<a href="https://colab.research.google.com/github/JiNathan/StockMarketPrediction/blob/main/News_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Data Processing
## Imports

In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google'

In [ ]:
import datetime
fpath = "/content/drive/My Drive/cnbc_news_datase.csv"

news = pd.read_csv(fpath, usecols = [1, 3, 6,  7, 10])
news.head(100) # this shows format of data



,title,published_at,short_description,keywords,description
0,Santoli’s Wednesday market notes: Could Septem...,2021-09-29T17:09:39+0000,"This is the daily notebook of Mike Santoli, CN...","cnbc, Premium, Articles, Investment strategy, ...","This is the daily notebook of Mike Santoli, CN..."
1,My take on the early Brexit winners and losers,2016-06-24T13:50:48-0400,This commentary originally ran on Facebook. Bo...,"Articles, Politics, Europe News, European Cent...",NaN
2,Europe&#039;s recovery depends on Renzi&#039;s...,2014-03-25T13:29:45-0400,"In spring, ambitious reforms began in Italy. U...","Articles, Business News, Economy, Europe Econo...",NaN
3,US Moves Closer to Becoming A Major Shareholde...,2009-04-22T19:49:03+0000,The US government is increasingly likely to co...,"cnbc, Articles, General Motors Co, Business Ne...",The US government is increasingly likely to co...
4,Trump: 'Mission accomplished' on 'perfectly ex...,2018-04-14T14:59:04+0000,NaN,"cnbc, Articles, George W. Bush, Vladimir Putin...",President Donald Trump hailed the U.S.-led int...
...,...,...,...,...,...
95,GLOBAL MARKETS-Euro rises on Spain speculation...,2012-10-02T18:23:00+0000,"(Adds comment, details, updates prices)* Spain...","cnbc, Articles, Caterpillar Inc, Europe, Washi...","(Adds comment, details, updates prices)* Spain..."
96,"'I come to bury Bitcoin, not to praise it': UBS",2018-11-30T11:38:30+0000,Cryptocurrencies are nearing the end of the ro...,"cnbc, Articles, Bitcoin/USD Bitfinex, Economy,...",Cryptocurrencies are nearing the end of the ro...
97,Jon Stewart joins Stephen Colbert to mock that...,2016-07-22T11:44:12+0000,It's been 351 days since Jon Stewart sat behin...,"cnbc, Articles, Donald Trump, Media, Elections...",It's been 351 days since Jon Stewart sat behin...
98,Will Stocks Resist 'Anything but Utmost Catast...,2011-11-17T11:50:06+0000,Stock markets have taken such a beating over t...,"cnbc, Articles, Business News, Economy, World ...",Stock markets have taken such a beating over t...


In [ ]:
def toTime(date_string):
    date = datetime.datetime(int(date_string[:4]), int(date_string[5:7]), int(date_string[8:10]))
    return date

class NewsArticle: # Class to represent and manipulate NewsArticle objects
  def __init__(self, id, title, date, keywords, description, short_description):
    self.id = id
    self.title = title
    self.keywords = keywords.split(", ")
    self.date = toTime(date)
    self.description = description
    self.short_description = short_description

  def __str__(self):
    return str(self.id) + ": " + str(self.title) + ": " + str(self.short_description) + "(" + str(self.date) + ")"

  def __repr__(self):
    return str(self.title) + ": " + str(self.description) + "(" + str(self.date) + ")" + "; keywords: " + str(self.keywords)

  def add_keyword(self, keyword):
    '''adds a keyword to the list of the article's keywords'''
    self.keywords.append(keyword)

  def contains_keyword(self, keyword_to_find):
    '''checks if article contains given keyword'''
    for keyword in self.keywords:
      if keyword_to_find in keyword:
        return True;
    return False;


In [ ]:
articles = []
for j in range(len(news)): # process news article objects
  article = NewsArticle(j, news.iloc[j][0], news.iloc[j][1], news.iloc[j][3], news.iloc[j][4], news.iloc[j][2])
  articles.append(article)

#for i in range(50):
  #print(articles[i])

def find_relevant_articles(article_list, keyword, range_start=0, range_end=500):
  '''function to find relevant articles for a given keyword'''
  if (range_start < 0):
    return "Invalid start index"
  relevant_articles = []
  for i in range(range_start, min(range_end, len(article_list))):
    if (article_list[i].contains_keyword(keyword)):
      relevant_articles.append(article_list[i])
  return relevant_articles

# demos
print("Articles about Donald Trump:")
ex_list = find_relevant_articles(articles, "Donald Trump", 0, 200)
for article in ex_list:
  print(article)
print("\nArticles about Bitcoin:")
ex_list = find_relevant_articles(articles, "Bitcoin", 0, 300)
for article in ex_list:
  print(article)

Articles about Donald Trump:
4: Trump: 'Mission accomplished' on 'perfectly executed' Syria strike: nan(2018-04-14 00:00:00)
5: Chevron CEO Watson says he supports Trump on tax reform: Chevron Chief Executive John Watson told CNBC Tuesday that he "strongly supports U.S. tax reform" under President Donald Trump's administration. Trump's administration wants to make U.S. tax reform competitive, Watson said during an interview on CNBC's "Closing Bell." Specifically he discussed a proposed bill — a border-adjustment tax — that could hike rates on American imports. Despite concerns that a border tax could hurt oil prices, Chevron's Watson said industry imports would "come into balance" over time. "The unpredictable effects of the U.S. dollar strengthening is what concerns a lot of people," he explained.(2017-03-07 00:00:00)
71: House Foreign Affairs chairman blasts Trump administration for report on Soleimani killing: The chairman of the House Foreign Affairs Committee on Friday blasted the